In [31]:
import tensorflow as tf
import numpy as np
import os

In [32]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [33]:
x_new1 = np.load('./checkpoints/x_newtrain1.npy')
x_new5 = np.load('./checkpoints/x_newtrain5.npy')
x_new10 = np.load('./checkpoints/x_newtrain10.npy')
x_new20 = np.load('./checkpoints/x_newtrain20.npy')
x_new30 = np.load('./checkpoints/x_newtrain30.npy')
x_new40 = np.load('./checkpoints/x_newtrain40.npy')
x_new50 = np.load('./checkpoints/x_newtrain50.npy')

In [4]:
# Parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 100
display_step = 1

In [5]:
# tf Graph Input
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

# Set model weights
W1 = tf.Variable(tf.random_normal([784, 300], mean=0, stddev=1))
b1 = tf.Variable(tf.random_normal([300], mean=0, stddev = 1))

#W2 = tf.Variable(tf.random_normal([300, 100], mean=0, stddev= 1))
#b2 = tf.Variable(tf.random_normal([100], mean=0, stddev= 1))

W3 = tf.Variable(tf.zeros([300, 10]))
b3 = tf.Variable(tf.zeros([10]))


In [6]:
# Construct model

hidden1 = tf.nn.relu(tf.matmul(x, W1) + b1); #first hidden layer

#hidden2 = tf.nn.relu(tf.matmul(hidden1, W2) + b2); #second hidden layer

pred = tf.nn.softmax(tf.matmul(hidden1, W3) + b3) # Softmax layer outputs prediction probabilities

In [7]:
# Minimize error using cross entropy 

cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))

In [8]:
# optimizer = {}
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# for name, fcn in cost:
#     optimizer["optimizer_{0}".format(name)] = tf.train.GradientDescentOptimizer(learning_rate).minimize(fcn)

In [9]:
x_comb1 = np.append(mnist.train.images, x_new1, axis = 0)
x_comb5 = np.append(mnist.train.images, x_new5, axis = 0)
x_comb10 = np.append(mnist.train.images, x_new10, axis = 0)
x_comb20 = np.append(mnist.train.images, x_new20, axis = 0)
x_comb30 = np.append(mnist.train.images, x_new30, axis = 0)
x_comb40 = np.append(mnist.train.images, x_new40, axis = 0)
x_comb50 = np.append(mnist.train.images, x_new50, axis = 0)

In [10]:
np.shape(mnist.train.images)

(55000, 784)

In [11]:
np.shape(x_comb1)

(110000, 784)

In [12]:
# y_comb = np.append(mnist.train.labels, mnist.test.labels[:3000], axis = 0)

y_comb = np.append(mnist.train.labels, mnist.train.labels, axis = 0)

In [13]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = data[ idx]
    labels_shuffle = labels[ idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

def attack_success_rate(xts, xts_new, yts):
        # Result of old test data
        prediction_old = tf.argmax(pred,1)
        prediction_old = prediction_old.eval({x: xts})    
        
        correct_prediction = tf.equal(prediction_old, tf.argmax(yts, 1))
        correct_prediction = correct_prediction.eval({x: xts})
        
        # Because we are only 
        correct_prediction_index = np.where(correct_prediction)
        
        xts_correct = xts_new[correct_prediction_index,:]
        xts_correct = xts_correct[0,:,:]
        
        correct_prediction = correct_prediction[correct_prediction_index]
        prediction_old = prediction_old[correct_prediction_index]
        
        # Result of new test data
        prediction_new = tf.argmax(pred,1)
        prediction_new = prediction_new.eval({x:xts_correct})
        
        #changed_prediction = tf.not_equal(prediction_old, prediction_new)
        #changed_prediction = changed_prediction.eval({x: xts_correct})
        
        # Vind out which index of correct_predictions are changed after perturb
        attack_success_index = np.not_equal(prediction_old, prediction_new)
        
        # Calculaye attack ratio
        attack_success_no = np.count_nonzero(attack_success_index)
        correct_prediction_no = np.count_nonzero(correct_prediction)
        
        attack_success_rate = attack_success_no/correct_prediction_no
        
        return attack_success_rate


saver = tf.train.Saver()
# save_dir = 'checkpoints/'
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)
# save_path = os.path.join(save_dir, 'trained_model')

In [14]:
# # coding: utf-8

# # Start training
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
    
#     # Training cycle
#     for epoch in range(training_epochs):
#         avg_cost = 0.
#         total_batch = int(np.shape(x_comb1)[0]/batch_size)
# #         print("number of total_batch: %"%s)
#         # Loop over all batches
#         for i in range(total_batch):
            
# #             batch_xs, batch_ys = next_batch(total_batch, x_comb1, y_comb)
# #             batch_xs, batch_ys = next_batch(total_batch, x_comb5, y_comb)
# #             batch_xs, batch_ys = next_batch(total_batch, x_comb10, y_comb)
#             batch_xs, batch_ys = next_batch(total_batch, x_comb20, y_comb)
# #             batch_xs, batch_ys = next_batch(total_batch, x_comb30, y_comb)
# #             batch_xs, batch_ys = next_batch(total_batch, x_comb40, y_comb)
# #             batch_xs, batch_ys = next_batch(total_batch, x_comb50, y_comb)
            
#             # Fit training using batch data
#             _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,y: batch_ys})
            
# #             print(__w)
            
#             # Compute average loss
#             avg_cost += c / total_batch
#         # Display logs per epoch step
#         if (epoch+1) % display_step == 0:
# #             print(sess.run(W))
#             print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

#     print ("Optimization Finished!")

#     # Test model
#     correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    
#     # Calculate accuracy for 3000 examples; you should get roughly ~90% accuracy although it might vary from run to run
#     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     print ("Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))
    
    
#     #save weights of the pertubed model
# #     saver.save(sess, './checkpoints/trained_model_p1.ckpt')
# #     saver.save(sess, './checkpoints/trained_model_p5.ckpt')
# #     saver.save(sess, './checkpoints/trained_model_p10.ckpt')
#     saver.save(sess, './checkpoints/trained_model_p20.ckpt')
# #     saver.save(sess, './checkpoints/trained_model_p30.ckpt')
# #     saver.save(sess, './checkpoints/trained_model_p40.ckpt')
# #     saver.save(sess, './checkpoints/trained_model_p50.ckpt')

#     print("Model saved")
    
    



In [15]:
saver = tf.train.Saver()
with tf.Session() as sess:
    
    print("Original model(no retraining).")
    saver.restore(sess, "./checkpoints/trained_model.ckpt")
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    
    # Calculate accuracy for 3000 examples; you should get roughly ~90% accuracy although it might vary from run to run
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     print ("Accuracy of original train dataset:", accuracy.eval({x: mnist.train.images, y: mnist.train.labels}))
    print ("Accuracy of original test dataset(3000):", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))
    print ("Accuracy of perturbed(eps = 1) train dataset:", accuracy.eval({x: x_new1, y: mnist.train.labels}))
    print ("Accuracy of perturbed(eps = 5) train dataset:", accuracy.eval({x: x_new5, y: mnist.train.labels}))
    print ("Accuracy of perturbed(eps = 10) train dataset:", accuracy.eval({x: x_new10, y: mnist.train.labels}))
    print ("Accuracy of perturbed(eps = 20) train dataset:", accuracy.eval({x: x_new20, y: mnist.train.labels}))
    print ("Accuracy of perturbed(eps = 30) train dataset:", accuracy.eval({x: x_new30, y: mnist.train.labels}))
    print ("Accuracy of perturbed(eps = 40) train dataset:", accuracy.eval({x: x_new40, y: mnist.train.labels}))
    print ("Accuracy of perturbed(eps = 50) train dataset:", accuracy.eval({x: x_new50, y: mnist.train.labels}))
    print ("Accuracy of mixed normal & perturbed FGSM dataset:", accuracy.eval({x: np.append(x_new10, mnist.train.images, axis = 0), 
                                       y: np.append(mnist.train.labels, mnist.train.labels, axis = 0)}))

Original model(no retraining).
INFO:tensorflow:Restoring parameters from ./checkpoints/trained_model.ckpt
Accuracy of original test dataset(3000): 0.902333
Accuracy of perturbed(eps = 1) train dataset: 0.897545
Accuracy of perturbed(eps = 5) train dataset: 0.647691
Accuracy of perturbed(eps = 10) train dataset: 0.207218
Accuracy of perturbed(eps = 20) train dataset: 0.00298182
Accuracy of perturbed(eps = 30) train dataset: 0.000127273
Accuracy of perturbed(eps = 40) train dataset: 7.27273e-05
Accuracy of perturbed(eps = 50) train dataset: 0.0
Accuracy of mixed normal & perturbed FGSM dataset: 0.568655


In [16]:
saver = tf.train.Saver()
with tf.Session() as sess:
    
    print("Retrained eps = 1 model")
    saver.restore(sess, "./checkpoints/trained_model_p1.ckpt")
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    
    # Calculate accuracy for 3000 examples; you should get roughly ~90% accuracy although it might vary from run to run
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     print ("Accuracy of original train dataset:", accuracy.eval({x: mnist.train.images, y: mnist.train.labels}))
    print ("Accuracy of original test dataset(3000):", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))
    print ("Accuracy of perturbed(eps = 1) train dataset:", accuracy.eval({x: x_new1, y: mnist.train.labels}))
    print ("Accuracy of mixed normal & perturbed FGSM dataset:", accuracy.eval({x: np.append(x_new1, mnist.train.images, axis = 0), 
                                       y: np.append(mnist.train.labels, mnist.train.labels, axis = 0)}))

Retrained eps = 1 model
INFO:tensorflow:Restoring parameters from ./checkpoints/trained_model_p1.ckpt
Accuracy of original test dataset(3000): 0.909667
Accuracy of perturbed(eps = 1) train dataset: 0.933873
Accuracy of mixed normal & perturbed FGSM dataset: 0.934973


In [30]:
saver = tf.train.Saver()
with tf.Session() as sess:
    
    print("Retrained eps = 5 model")
    saver.restore(sess, "./checkpoints/trained_model_p5.ckpt")
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    
    # Calculate accuracy for 3000 examples; you should get roughly ~90% accuracy although it might vary from run to run
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     print ("Accuracy of original train dataset:", accuracy.eval({x: mnist.train.images, y: mnist.train.labels}))
    print ("Accuracy of original test dataset(3000):", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))
    print ("Accuracy of perturbed(eps = 5) train dataset:", accuracy.eval({x: x_new5, y: mnist.train.labels}))
    print ("Accuracy of mixed normal & perturbed FGSM dataset:", accuracy.eval({x: np.append(x_new5, mnist.train.images, axis = 0), 
                                       y: np.append(mnist.train.labels, mnist.train.labels, axis = 0)}))

Retrained eps = 5 model
INFO:tensorflow:Restoring parameters from ./checkpoints/trained_model_p5.ckpt
Accuracy of original test dataset(3000): 0.912333
Accuracy of perturbed(eps = 5) train dataset: 0.935291
Accuracy of mixed normal & perturbed FGSM dataset: 0.935609


In [29]:
saver = tf.train.Saver()
with tf.Session() as sess:
    
    print("Retrained eps = 10 model")
    saver.restore(sess, "./checkpoints/trained_model_p10.ckpt")
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    
    # Calculate accuracy for 3000 examples; you should get roughly ~90% accuracy although it might vary from run to run
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     print ("Accuracy of original train dataset:", accuracy.eval({x: mnist.train.images, y: mnist.train.labels}))
    print ("Accuracy of original test dataset(3000):", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))
    print ("Accuracy of perturbed(eps = 10) train dataset:", accuracy.eval({x: x_new10, y: mnist.train.labels}))
    print ("Accuracy of mixed normal & perturbed FGSM dataset:", accuracy.eval({x: np.append(x_new10, mnist.train.images, axis = 0), 
                                       y: np.append(mnist.train.labels, mnist.train.labels, axis = 0)}))

Retrained eps = 10 model
INFO:tensorflow:Restoring parameters from ./checkpoints/trained_model_p10.ckpt
Accuracy of original test dataset(3000): 0.905667
Accuracy of perturbed(eps = 10) train dataset: 0.943164
Accuracy of mixed normal & perturbed FGSM dataset: 0.935855


In [28]:
saver = tf.train.Saver()
with tf.Session() as sess:
    
    print("Retrained eps = 20 model")
    saver.restore(sess, "./checkpoints/trained_model_p20.ckpt")
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    
    # Calculate accuracy for 3000 examples; you should get roughly ~90% accuracy although it might vary from run to run
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     print ("Accuracy of original train dataset:", accuracy.eval({x: mnist.train.images, y: mnist.train.labels}))
    print ("Accuracy of original test dataset(3000):", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))
    print ("Accuracy of perturbed(eps = 20) train dataset:", accuracy.eval({x: x_new20, y: mnist.train.labels}))
    print ("Accuracy of mixed normal & perturbed FGSM dataset:", accuracy.eval({x: np.append(x_new20, mnist.train.images, axis = 0), 
                                       y: np.append(mnist.train.labels, mnist.train.labels, axis = 0)}))

Retrained eps = 20 model
INFO:tensorflow:Restoring parameters from ./checkpoints/trained_model_p20.ckpt
Accuracy of original test dataset(3000): 0.890333
Accuracy of perturbed(eps = 20) train dataset: 0.967855
Accuracy of mixed normal & perturbed FGSM dataset: 0.943464


In [20]:
saver = tf.train.Saver()
with tf.Session() as sess:
    
    print("Retrained eps = 30 model")
    saver.restore(sess, "./checkpoints/trained_model_p30.ckpt")
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    
    # Calculate accuracy for 3000 examples; you should get roughly ~90% accuracy although it might vary from run to run
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     print ("Accuracy of original train dataset:", accuracy.eval({x: mnist.train.images, y: mnist.train.labels}))
    print ("Accuracy of original test dataset(3000):", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))
    print ("Accuracy of perturbed(eps = 30) train dataset:", accuracy.eval({x: x_new30, y: mnist.train.labels}))
    print ("Accuracy of mixed normal & perturbed FGSM dataset:", accuracy.eval({x: np.append(x_new30, mnist.train.images, axis = 0), 
                                       y: np.append(mnist.train.labels, mnist.train.labels, axis = 0)}))

Retrained eps = 30 model
INFO:tensorflow:Restoring parameters from ./checkpoints/trained_model_p30.ckpt
Accuracy of original test dataset(3000): 0.884667
Accuracy of perturbed(eps = 30) train dataset: 0.761982
Accuracy of mixed normal & perturbed FGSM dataset: 0.838391


In [27]:
saver = tf.train.Saver()
with tf.Session() as sess:
    
    print("Retrained eps = 40 model")
    saver.restore(sess, "./checkpoints/trained_model_p40.ckpt")
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    
    # Calculate accuracy for 3000 examples; you should get roughly ~90% accuracy although it might vary from run to run
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     print ("Accuracy of original train dataset:", accuracy.eval({x: mnist.train.images, y: mnist.train.labels}))
    print ("Accuracy of original test dataset(3000):", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))
    print ("Accuracy of perturbed(eps = 40) train dataset:", accuracy.eval({x: x_new40, y: mnist.train.labels}))
    print ("Accuracy of mixed normal & perturbed FGSM dataset:", accuracy.eval({x: np.append(x_new40, mnist.train.images, axis = 0), 
                                       y: np.append(mnist.train.labels, mnist.train.labels, axis = 0)}))

Retrained eps = 40 model
INFO:tensorflow:Restoring parameters from ./checkpoints/trained_model_p40.ckpt
Accuracy of original test dataset(3000): 0.886333
Accuracy of perturbed(eps = 40) train dataset: 0.984073
Accuracy of mixed normal & perturbed FGSM dataset: 0.948273


In [34]:
saver = tf.train.Saver()
with tf.Session() as sess:
    
    print("Retrained eps = 50 model")
    saver.restore(sess, "./checkpoints/trained_model_p50.ckpt")
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    
    # Calculate accuracy for 3000 examples; you should get roughly ~90% accuracy although it might vary from run to run
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     print ("Accuracy of original train dataset:", accuracy.eval({x: mnist.train.images, y: mnist.train.labels}))
    print ("Accuracy of original test dataset(3000):", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))
    print ("Accuracy of perturbed(eps = 50) train dataset:", accuracy.eval({x: x_new50, y: mnist.train.labels}))
    print ("Accuracy of mixed normal & perturbed FGSM dataset:", accuracy.eval({x: np.append(x_new50, mnist.train.images, axis = 0), 
                                       y: np.append(mnist.train.labels, mnist.train.labels, axis = 0)}))

Retrained eps = 50 model
INFO:tensorflow:Restoring parameters from ./checkpoints/trained_model_p50.ckpt
Accuracy of original test dataset(3000): 0.889667
Accuracy of perturbed(eps = 50) train dataset: 0.497127
Accuracy of mixed normal & perturbed FGSM dataset: 0.7072


In [23]:

# # saver = tf.train.Saver()

# with tf.Session() as sess:
#   # Restore variables from disk.
#   saver.restore(sess, "/tmp/model.ckpt")
#   print("Model restored.")